In [1]:
import os, datetime, time, random, json, warnings
import gensim
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type

In [2]:
directory_path = "\\".join(os.getcwd().split("\\")[:-1]) + "\\data"
os.path.exists(directory_path)

True

In [3]:
class TrainingConfig(object):
    epochs = 5
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001


class ModelConfig(object):
    embeddingSize = 200
    hiddenSizes = 128
    dropoutKeepProb = 0.5
    leRegLambda = 0.0
    epsilon = 5

    
class Config(object):
    sequenceLength = 200
    batchSize = 128
    dataSource = directory_path + "\\preProcess\\labeledTrain.csv"
    stopWordSource = directory_path + "\\english"
    
    # 二分类设置为 1，多分类设置为其他数字
    numClasses = 1
    rate = 0.8
    training = TrainingConfig()
    model = ModelConfig()

    
config = Config()

In [5]:
class Dataset(object):
    def __init__(self, config):
        self.config = config
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource
        self._sequenceLength = config.sequenceLength
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = dict()
        self.trainReviews = []
        self.trainLabels = []
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding = None
        
        # 统计词在多少个 review 中出现过
        self.indexFreqs = []
        self.labelList = []
    
    
    def _readData(self, file_path):
        df = pd.read_csv(file_path)
        if self.config.numClasses == 1:
            labels = df["sentiment"].tolist()
        elif self.config.numClasses > 1:
            labels =df["rate"].tolist()
        
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]
        
        return reviews, labels
    
    
    def _labelToIndex(self, labels, label2idx):
        labelIds = [label2idx[label] for label in labels]
        return labelIds
    
    
    def _wordToIndex(self, reviews, word2idx):
        reviewIds = [[word2idx.get(item, word2idx["UNK"]) for item in review] for review in reviews]
        return reviewIds
    
    
    def _genTrainEvalData(self, x, y, word2idx, rate):
        reviews = []
        for review in x:
            if len(review) >= self._sequenceLength:
                reviews.append(review[:self._sequenceLength])
            else:
                reviews.append(review + [word2idx["PAD"]] * (self._sequenceLength - len(review)))
        
        trainIndex = int(len(x) * rate)
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(y[:trainIndex], dtype="float32")
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels= np.array(y[trainIndex:], dtype="float32")
        
        return trainReviews, trainLabels, evalReviews, evalLabels
    
    
    
    def _getWordEmbedding(self, words):
        wordVec = gensim.models.KeyedVectors.load_word2vec_format(directory_path + "\\word2vec\\word2Vec.bin", 
                                                                 binary=True)
        vocab = []
        wordEmbedding = []
        vocab.append("PAD")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print("{} is not exist...")
        
        return vocab, wordEmbedding
    
    
    
    def _getWordIndexFreqs(self, vocab, reviews, word2idx):
        """
            本函数旨在统计词汇空间内各个词出现在多少个文本中，亦即：如"科技"出现在多少个review中
        """
        reviewDicts = [dict(zip(review, range(len(review)))) for review in reviews]
        indexFreqs = [0] * len(vocab)
        
        start_time = datetime.datetime.now().isoformat()
        for word in vocab:
            count = 0
            for review in reviews:
                if word in set(review):
                    count += 1
                
            indexFreqs[word2idx[word]] = count
        end_time = datetime.datetime.now().isoformat()
        print("words index freqd compute start with: {}, end with: {}".format(start_time, end_time))
        
        self.indexFreqs = indexFreqs
      
    
    def _genVocabulary(self, reviews, labels):
        all_words = [word for review in reviews for word in review]
        subWords = [word for word in all_words if word not in self.stopWordDict]
        wordCount = Counter(subWords)
        sortedWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        words = [item[0] for item in sortedWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        word2idx = dict(zip(vocab, list(range(len(vocab)))))
        self._getWordIndexFreqs(vocab, reviews, word2idx)
        
        uniqueLabel = list(set(labels))
        label2idx = dict(zip(uniqueLabel, list(range(len(uniqueLabel)))))
        self.labelList = list(range(len(uniqueLabel)))
        
        with open(directory_path + "\\wordJson\\word2idx.json", "w", encoding="utf-8") as f:
            json.dump(word2idx, f)
        with open(directory_path + "\\wordJson\\label2idx.json", "w", encoding="utf-8") as f:
            json.dump(label2idx, f)
        
        return word2idx, label2idx
        
    
    
    def _readStopWord(self, stopWordPath):
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    
    
    def dataGen(self):
        self._readStopWord(self._stopWordSource)
        reviews, labels = self._readData(self._dataSource)
        word2idx, label2idx = self._genVocabulary(reviews, labels)
        labelIds = self._labelToIndex(labels, label2idx)
        reviewIds = self._wordToIndex(reviews, word2idx)
        
        self.trainReviews, self.trainLabels, self.evalReviews, self.evalLabels = self._genTrainEvalData(reviewIds, 
                                                                                                       labelIds, 
                                                                                                       word2idx, 
                                                                                                       self._rate)
    
data = Dataset(config)
data.dataGen()

words index freqd compute start with: 2019-09-26T18:58:24.416667, end with: 2019-09-26T21:45:30.625821


In [6]:
def nextBatch(x,y, batchSize):
    perm = np.arange(len(x))
    np.random.shuffle(perm)
    x = x[perm]
    y = y[perm]
    numBatches = len(x) // batchSize
    
    for index in range(numBatches):
        start = index * batchSize
        end = start + batchSize
        batchX = np.array(x[start: end], dtype="int64")
        batchY = np.array(y[start: end], dtype="float32")
        
        yield batchX, batchY
        

In [7]:
def mean(item: list) -> float:
    """
    计算列表中元素的平均值
    :param item: 列表对象
    :return:
    """
    res = sum(item) / len(item) if len(item) > 0 else 0
    return res


def accuracy(pred_y, true_y):
    """
    计算二类和多类的准确率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]
    corr = 0
    for i in range(len(pred_y)):
        if pred_y[i] == true_y[i]:
            corr += 1
    acc = corr / len(pred_y) if len(pred_y) > 0 else 0
    return acc


def binary_precision(pred_y, true_y, positive=1):
    """
    二类的精确率计算
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param positive: 正例的索引表示
    :return:
    """
    corr = 0
    pred_corr = 0
    for i in range(len(pred_y)):
        if pred_y[i] == positive:
            pred_corr += 1
            if pred_y[i] == true_y[i]:
                corr += 1

    prec = corr / pred_corr if pred_corr > 0 else 0
    return prec


def binary_recall(pred_y, true_y, positive=1):
    """
    二类的召回率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param positive: 正例的索引表示
    :return:
    """
    corr = 0
    true_corr = 0
    for i in range(len(pred_y)):
        if true_y[i] == positive:
            true_corr += 1
            if pred_y[i] == true_y[i]:
                corr += 1

    rec = corr / true_corr if true_corr > 0 else 0
    return rec


def binary_f_beta(pred_y, true_y, beta=1.0, positive=1):
    """
    二类的f beta值
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param beta: beta值
    :param positive: 正例的索引表示
    :return:
    """
    precision = binary_precision(pred_y, true_y, positive)
    recall = binary_recall(pred_y, true_y, positive)
    try:
        f_b = (1 + beta * beta) * precision * recall / (beta * beta * precision + recall)
    except:
        f_b = 0
    return f_b

def multi_precision(pred_y, true_y, labels):
    """
    多类的精确率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param labels: 标签列表
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]

    precisions = [binary_precision(pred_y, true_y, label) for label in labels]
    prec = mean(precisions)
    return prec


def multi_recall(pred_y, true_y, labels):
    """
    多类的召回率
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param labels: 标签列表
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]

    recalls = [binary_recall(pred_y, true_y, label) for label in labels]
    rec = mean(recalls)
    return rec


def multi_f_beta(pred_y, true_y, labels, beta=1.0):
    """
    多类的f beta值
    :param pred_y: 预测结果
    :param true_y: 真实结果
    :param labels: 标签列表
    :param beta: beta值
    :return:
    """
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]

    f_betas = [binary_f_beta(pred_y, true_y, beta, label) for label in labels]
    f_beta = mean(f_betas)
    return f_beta


def get_binary_metrics(pred_y, true_y, f_beta=1.0):
    """
    得到二分类的性能指标
    :param pred_y:
    :param true_y:
    :param f_beta:
    :return:
    """
    acc = accuracy(pred_y, true_y)
    recall = binary_recall(pred_y, true_y)
    precision = binary_precision(pred_y, true_y)
    f_beta = binary_f_beta(pred_y, true_y, f_beta)
    return acc, recall, precision, f_beta


def get_multi_metrics(pred_y, true_y, labels, f_beta=1.0):
    """
    得到多分类的性能指标
    :param pred_y:
    :param true_y:
    :param labels:
    :param f_beta:
    :return:
    """
    acc = accuracy(pred_y, true_y)
    recall = multi_recall(pred_y, true_y, labels)
    precision = multi_precision(pred_y, true_y, labels)
    f_beta = multi_f_beta(pred_y, true_y, labels, f_beta)
    return acc, recall, precision, f_beta

In [46]:
class AdversarialLSTM(object):
    def __init__(self, config, wordEmbedding, indexFreqs):
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.int32, [None], name="inputY")
        self.dropoutKeepProb= tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.config = config
        
        # 根据词频计算权重
#         print("the index of frequence is: ", indexFreqs)
        indexFreqs[0], indexFreqs[1] = 20000, 10000
        weights = tf.cast(tf.reshape(indexFreqs / tf.reduce_sum(indexFreqs), [1, len(indexFreqs)]), dtype=tf.float32)
        
#         print("the value of weights: ", weights)
#         print("the value of wordEmbedding: ", wordEmbedding)
        
        
        
        # word-embedding layer
        with tf.name_scope("embedding"):
            # 利用词频计算新的 word-embedding matrix
            normWordEmbedding = self._normalize(tf.cast(np.array(wordEmbedding), dtype=tf.float32, name="word2vec"), weights)
            # 利用 word-embedding matrix 将输入的数据中的词转换成 word-vector，维度为：
            # [batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(normWordEmbedding, self.inputX)
        
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.logits = self._Bi_LSTMAttention(self.embeddedWords)
                if config.numClasses == 1:
                    self.predictions = tf.cast(tf.greater_equal(self.logits, 0.0), tf.float32, name="predictions")
                    losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, 
                                                                    labels=tf.cast(tf.reshape(self.inputY, [-1, 1]), 
                                                                                  dtype=tf.float32))
                elif config.numClasses > 1:
                    self.predictions = tf.argmax(self.logits, axis=-1, name="predictions")
                    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.inputY)
                
                loss = tf.reduce_mean(losses)
       
        
        with tf.name_scope("perturLoss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, 
                                                                      labels=tf.cast(tf.reshape(self.inputY, [-1, 1]), 
                                                                                    dtype=tf.float32))
                perturLoss = tf.reduce_mean(perturLosses)
        
        self.loss = loss + perturLoss
    
    
    
    def _Bi_LSTMAttention(self, embeddedWords):
        config = self.config
        with tf.name_scope("Bi-LSTM"):
            lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, 
                                                                              state_is_tuple=True), 
                                                      output_keep_prob=self.dropoutKeepProb)
            lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, 
                                                                              state_is_tuple=True), 
                                                      output_keep_prob=self.dropoutKeepProb)
            
            outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                          self.embeddedWords, dtype=tf.float32, 
                                                                          scope="bi-lstm")
        
        # 论文中是将前向和后向相加
        with tf.name_scope("Attention"):
            H = outputs_[0] + outputs_[1]
            output = self._attention(H)
            outputSize = config.model.hiddenSizes
        
        # FC layer
        with tf.name_scope("output"):
            outputW = tf.get_variable("outputW", shape=[outputSize, config.numClasses], 
                                      initializer=tf.contrib.layers.xavier_initializer())
            outputB = tf.Variable(tf.constant(0.1, shape=[config.numClasses]), name="outputB")
            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
        
        
        return predictions
    
    
    def _attention(self, H):
        hiddenSize = config.model.hiddenSizes
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        M = tf.tanh(H)
        
        # newM shape = [batch_size, time_step, 1]
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # restoreM shape = [batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        self.alpha = tf.nn.softmax(restoreM)
        
        # 针对alpha对H进行加权
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成为二维[batch_size, time_step]
        sequeezeR = tf.squeeze(r)
        sentence_representation = tf.tanh(sequeezeR)
        output = tf.nn.dropout(sentence_representation, self.dropoutKeepProb)
        
        return output
            
    
    
    
    def _normalize(self, wordEmbedding, weights):
        """
            对 wordEmbedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
#         print("the weight mean of wordEmbedding: ", mean)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)
        
        var = tf.matmul(weights, powWordEmbedding)
#         print("the weight var of powWordEmbedding is: ", var)
        stddev = tf.sqrt(1e-6 + var)
        
        return (wordEmbedding - mean) / stddev
    
    
    def _addPerturbation(self, embedded, loss):
        """
            在原始的word-embedding中添加扰动
        """
        grad, = tf.gradients(loss, embedded, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, self.config.model.epsilon)
    
        return embedded + perturb
    
    
    
    def _scaleL2(self, x, norm_length):
        # x.shape = [batch, num_time_steps, dim]
        alpha =tf.reduce_max(tf.abs(x), (1, 2), keepdims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keepdims=True) + 1e-6)
        x_unit = x / l2_norm
        
        return norm_length * x_unit
    
        

In [47]:
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels
wordEmbedding = data.wordEmbedding
indexFreqs = data.indexFreqs
labelList = data.labelList

In [63]:
with tf.Graph().as_default():
    sess_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=sess_config)
    
    with sess.as_default():
        lstm = AdversarialLSTM(config, wordEmbedding, indexFreqs)
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summary"))
        print("write to {} \n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter =tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        savedModelPath = "\\".join(directory_path.split("\\")[:-1]) + "\\model\\adversarialLSTM\\savedModel"
        if os.path.exists(savedModelPath):
#             os.rmdir(savedModelPath)
            import shutil
            shutil.rmtree(savedModelPath)
        
#         builder = tf.saved_model.builder.SavedModelBuilder(savedModelPath)
        sess.run(tf.global_variables_initializer())
        
        def trainStep(batchX, batchY):
            feed_dict = {
                lstm.inputX: batchX, 
                lstm.inputY: batchY, 
                lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions = sess.run([trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions], 
                                                          feed_dict)
            
            if config.numClasses == 1:
                acc, recall, prec, f_beta = get_binary_metrics(pred_y=predictions, true_y=batchY)
            elif config.numClasses > 1:
                acc, recall, prec, f_beta = get_multi_metrics(pred_y=predictions, true_y=batchY, labels=labelList)
            
            trainSummaryWriter.add_summary(summary, step)
            
            return loss, acc, prec, recall, f_beta
        
        
        def devStep(batchX, batchY):
            feed_dict = {
                lstm.inputX: batchX, 
                lstm.inputY: batchY, 
                lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions = sess.run([summaryOp, globalStep, lstm.loss, lstm.predictions], 
                                                       feed_dict)
            
            if config.numClasses == 1:
                acc, recall, prec, f_beta = get_binary_metrics(pred_y=predictions, true_y=batchY)
            elif config.numClasses > 1:
                acc, recall, prec, f_beta = get_multi_metrics(pred_y=predictions, true_y=batchY, labels=labelList)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, prec, recall, f_beta
        
        
        for index in range(config.training.epochs):
            print("begin to train model...")
            print("the {}-th / {} epoch".format(str(index), str(config.training.epochs)))
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                loss, acc, prec, recall, f_beta = trainStep(batchTrain[0], batchTrain[1])
                currentStep = tf.train.global_step(sess, globalStep)
                timeStr = datetime.datetime.now().isoformat()
                
                print("training: time: {}, step: {}, loss: {}, acc: {}, recall: {}, precision: {}, f_beta: {}".format(
                    timeStr, currentStep, loss, acc, recall, prec, f_beta))
                if currentStep % config.training.evaluateEvery == 0:
                    print("begin to evaluation: \n")
                    losses, accs, recalls, precisions, f_betas = [], [], [], [], []
                    for evalBatch in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, precision, recall, f_beta = devStep(evalBatch[0], evalBatch[1])
                        losses.append(loss)
                        accs.append(acc)
                        recalls.append(recall)
                        precisions.append(precision)
                        f_betas.append(f_beta)
                    
                    time_str = datetime.datetime.now().isoformat()
                    print("evaluation: time: {}, step: {}, loss: {}, acc: {}, recall: {}, precision: {}, f_beta: {}".format(
                        time_str, currentStep, mean(losses), mean(accs), mean(recalls), mean(precisions), mean(f_betas)))
                
                if currentStep % config.training.checkpointEvery == 0:
                    path = saver.save(sess, "\\".join(savedModelPath.split("\\")[:-1]) + "\\model\\adversarialLSTM", 
                                     global_step=currentStep)
                    print("save model checkpoint to {} \n".format(path))
                
#                 inputs = {
#                     "inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX), 
#                     "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)
#                 }
#                 outputs = {"predictions": tf.saved_model.utils.build_tensor_info(lstm.predictions)}
                
#                 prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, 
#                                                                                              outputs=outputs, 
#                                                                                              method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
                
#                 legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#                 builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING], 
#                                                     signature_def_map={"predict": prediction_signature, }, 
#                                                     legacy_init_op=legacy_init_op)

# #                 builder.add_meta_graph(sess, [tf.saved_model.tag_constants.SERVING], 
# #                                                     signature_def_map={"predict": prediction_signature, }, 
# #                                                     legacy_init_op=legacy_init_op)
                
#                 builder.save()     

write to C:\Users\123\Documents\python_experence\nlp_model\model_code\summary 

begin to train model...
the 0-th / 5 epoch
training: time: 2019-09-27T11:27:05.772773, step: 1, loss: 1.5065041780471802, acc: 0.515625, recall: 0.9230769230769231, precision: 0.5128205128205128, f_beta: 0.6593406593406593
training: time: 2019-09-27T11:27:09.458243, step: 2, loss: 2.1197540760040283, acc: 0.4921875, recall: 0.0, precision: 0, f_beta: 0
training: time: 2019-09-27T11:27:13.345452, step: 3, loss: 1.64058256149292, acc: 0.5, recall: 0.015384615384615385, precision: 1.0, f_beta: 0.030303030303030307
training: time: 2019-09-27T11:27:17.736116, step: 4, loss: 1.352006196975708, acc: 0.5859375, recall: 0.4915254237288136, precision: 0.5576923076923077, f_beta: 0.5225225225225225
training: time: 2019-09-27T11:27:21.861287, step: 5, loss: 1.4735063314437866, acc: 0.546875, recall: 0.8805970149253731, precision: 0.5412844036697247, f_beta: 0.6704545454545454
training: time: 2019-09-27T11:27:25.738556,

W0927 12:16:56.310768 17616 deprecation.py:323] From d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


save model checkpoint to C:\Users\123\Documents\python_experence\nlp_model\model\adversarialLSTM\model\adversarialLSTM-600 

training: time: 2019-09-27T12:17:02.303243, step: 601, loss: 0.6239484548568726, acc: 0.8359375, recall: 0.9117647058823529, precision: 0.8051948051948052, f_beta: 0.8551724137931035
training: time: 2019-09-27T12:17:06.231912, step: 602, loss: 0.5308969020843506, acc: 0.8515625, recall: 0.8840579710144928, precision: 0.8472222222222222, f_beta: 0.8652482269503546
training: time: 2019-09-27T12:17:10.273605, step: 603, loss: 0.48731115460395813, acc: 0.9140625, recall: 0.9452054794520548, precision: 0.9078947368421053, f_beta: 0.9261744966442953
training: time: 2019-09-27T12:17:14.038483, step: 604, loss: 0.5149098634719849, acc: 0.890625, recall: 0.8833333333333333, precision: 0.8833333333333333, f_beta: 0.8833333333333333
training: time: 2019-09-27T12:17:17.704832, step: 605, loss: 0.4294637441635132, acc: 0.9140625, recall: 0.9245283018867925, precision: 0.875, 

In [ ]:
x = "this movie is full of references like mad max ii the wild one and many others the ladybug´s face it´s a clear reference or tribute to peter lorre this movie is a masterpiece we´ll talk much more about in the future"

with open(directory_path + "\\wordJson\\word2idx.json", "r", encoding="utf-8") as f:
    word2idx = json.load(f)

with open(directory_path + "\\wordJson\\label2idx.json", "r", encoding="utf-8") as f:
    label2idx = json.load(f)

idx2label = {value: key for key, value in label2idx.items()}
xIds = [word2idx.get(item, word2idx["UNK"]) for item in x.split(" ")]
if len(xIds) >= config.sequenceLength:
    xIds = xIds[:config.sequenceLength]
else:
    xIds = xIds + [word2idx["PAD"]] * (config.sequenceLength - len(xIds))

graph = tf.Graph()
with graph.as_default():
    sess_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=sess_config)
    
    with sess.as_default():
        local_path = "\\".join(savedModelPath.split("\\")[:-1]) + "\\model\\adversarialLSTM\\"
        checkpoint_file = tf.train.latest_checkpoint(local_path)
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)
        
        # 获取model的输入参数、输出的结果所依赖的输入值
        inputX = graph.get_operation_by_name("inputX").outputs[0]
        dropoutKeepProb = graph.get_operation_by_nmae("dropoutKeepProb").outputs[0]
        predictions = graph.get_tensor_by_name("output/predictions:0")
        pred = sess.run(predictions, feed_dict={
                inputX: [xIds], 
                dropoutKeepProb: 1.0
            })[0]

pred = [idx2label[item] for item in pred]
print("the prediction result is: ", pred)        